<a href="https://colab.research.google.com/github/evmpython/ERA5/blob/main/01_ERA5_GEE_mapa_TEMPERATURA_e_ANOMALIA_por_mes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


# Mapa de Precipitação e Anomalia para um Determinado `Mês` com os Dados do ERA-5


---


 - Dados utilizados: [ERA5](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_LAND_MONTHLY_AGGR?hl=pt-br)
 - Variável utilizada: Temperatura do ar em 2 m
 - Informações sobre os dados:
    - Resolução espacial: 11 km
    - Resolução temporal: mensal
    - Período dos dados: 1950-2025/02
    
 - Criar [conta](https://earthengine.google.com/) no Google Earth Engine. Exemplo: https://www.youtube.com/watch?v=NHhgaprxWRw
 - Realizado e atualizado: Enrique V. Mattos - 06/10/2025



---



# Preparando o ambiente

In [1]:
#=========================================================================#
#                   Inicilizando o GEE e XEE
#=========================================================================#
# instalando Xee
!pip install -q xee

# importa bibliotecas
import ee
import xarray as xr
import time

# inicializando o GEE
ee.Authenticate()
ee.Initialize(project='ee-enrique', opt_url='https://earthengine-highvolume.googleapis.com')

#=========================================================================#
#              Instalando bibliotecas e montando o drive
#=========================================================================#
# instalações
!pip install -q ultraplot cartopy salem rasterio pyproj geopandas

# importa bibliotecas
import ultraplot as uplt
import salem
import warnings
warnings.filterwarnings("ignore")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 102.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.1/86.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 110.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 41.4 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/ultraplot/__init__.py:34: UltraPlotWarning: Failed to write rc setting cftime.time_resolution_format = None. Must be None, bool, string, int, float, a list or tuple thereof, or a property cycler.
  from .config import *  # noqa: F401 F403
/usr/local/lib/python3.12/dist-packages/ultraplot/__init__.py:77: UltraPlotWarning: Rebuilding font cache. This usually happens after installing or updating ultraplot.
  register_fonts(default=True)


# Carregando os dados


In [2]:
%%time

# região de estudo
roi = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))
#roi = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(ee.Filter.eq('ADM1_NAME', 'Sao Paulo'))
#roi = ee.FeatureCollection('FAO/GAUL/2015/level2').filter(ee.Filter.eq('ADM2_NAME', 'Sao Paulo'))
#roi = ee.Geometry.Rectangle([-46.2009,-21.151596,-46.189293,-21.143079])

# função que transforma de metros para milímetros
def metros_to_milimetros(image):
    return image.multiply(1000.0)\
                .copyProperties(image, ['system:time_start'])\
                .set('date', image.date().format('YYYY-MM-dd'))

# carrega os dados de t2m
precip_mensal = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')\
               .select('total_precipitation_sum')\
               .filterDate('1990-01-01','2025-08-01')\
               .map(metros_to_milimetros)

# converte para Dataset
ds_precip_mensal = xr.open_dataset(precip_mensal,
                                engine = 'ee',
                                crs = 'EPSG:4326',
                                scale = 0.11,
                                geometry = roi.geometry())

# muda de "(time, lon, lat)" para "(time, lat, lon)"
ds_precip_mensal = ds_precip_mensal.transpose("time", "lat", "lon")

# mostrando os dados
ds_precip_mensal

CPU times: user 1.32 s, sys: 149 ms, total: 1.47 s
Wall time: 5.28 s


<xarray.Dataset> Size: 249MB
Dimensions:                  (time: 427, lon: 410, lat: 355)
Coordinates:
  * time                     (time) datetime64[ns] 3kB 1990-01-01 ... 2025-07-01
  * lon                      (lon) float64 3kB -73.93 -73.82 ... -29.05 -28.94
  * lat                      (lat) float64 3kB -33.7 -33.59 ... 5.135 5.245
Data variables:
    total_precipitation_sum  (time, lat, lon) float32 249MB ...
Attributes:
    crs:      EPSG:4326

# Calcula anomalia
- Demora 3min 58s.

In [ ]:
%%time
# define o "ANO" e "MÊS"
ano, mes = '2025', '07'
ano, mes = '2023', '12'

# nome dos meses
nome_mes = ['Janeiro', 'Fevereiro', 'Março', 'Abril', 'Maio', 'Junho', 'Julho', 'Agosto', 'Setembro', 'Outubro', 'Novembro', 'Dezembro']

# limites do Brasil
lonmin, lonmax, latmin, latmax = -75.0, -33.0, -35.0, 7.0

# leitura do shapefile do Brasil
shapefile_brasil = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/brasil/BRAZIL.shp')

# calculando a "CLIMATOLOGIA"
climatologia = ds_precip_mensal['total_precipitation_sum'].sel(time=slice('1990-01-01','2020-12-31')).groupby("time.month").mean("time")

# calculando a "ANOMALIA"
anomalia = ds_precip_mensal['total_precipitation_sum'].sel(time=slice(f'{ano}-{mes}',f'{ano}-{mes}')).groupby("time.month") - climatologia

# recorta o dado para região
anomalia = anomalia.salem.roi(shape=shapefile_brasil)

# recorta o dado para região
ds_precip_mensal = ds_precip_mensal.salem.roi(shape=shapefile_brasil)

CPU times: user 56.6 s, sys: 19.7 s, total: 1min 16s
Wall time: 2min 25s


In [ ]:
ds_precip_mensal

<xarray.Dataset> Size: 249MB
Dimensions:                  (time: 427, lat: 355, lon: 410)
Coordinates:
  * time                     (time) datetime64[ns] 3kB 1990-01-01 ... 2025-07-01
  * lon                      (lon) float64 3kB -73.93 -73.82 ... -29.05 -28.94
  * lat                      (lat) float64 3kB -33.7 -33.59 ... 5.135 5.245
Data variables:
    total_precipitation_sum  (time, lat, lon) float32 249MB nan nan ... nan nan
Attributes:
    crs:         EPSG:4326
    pyproj_srs:  +proj=longlat +datum=WGS84 +no_defs

In [ ]:
climatologia

<xarray.DataArray 'total_precipitation_sum' (month: 12, lat: 355, lon: 410)> Size: 7MB
array([[[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [ 70.94518 ,  70.033356,  67.14723 , ...,        nan,
                nan,        nan],
        [ 92.93816 ,  74.43258 ,  59.571434, ...,        nan,
                nan,        nan],
        [114.26918 ,  80.504   ,  51.65366 , ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
...
        [158.01178 , 152.4462  , 144.55331 , ...,        nan,
                nan,        nan],
        [186.97221 , 159.10622 , 136.01555 , ...,        nan,
                nan,        nan],
        [217.23595 , 168.50117 , 126.212166, ...,        nan,
                nan,        nan]],

       [[       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        [       nan,        nan,        nan, ...,        nan,
                nan,        nan],
        ...,
        [ 92.41408 ,  89.52986 ,  84.60044 , ...,        nan,
                nan,        nan],
        [118.09095 ,  97.212074,  80.036705, ...,        nan,
                nan,        nan],
        [143.78299 , 106.38589 ,  74.23079 , ...,        nan,
                nan,        nan]]], dtype=float32)
Coordinates:
  * lon      (lon) float64 3kB -73.93 -73.82 -73.71 ... -29.16 -29.05 -28.94
  * lat      (lat) float64 3kB -33.7 -33.59 -33.48 -33.37 ... 5.025 5.135 5.245
  * month    (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Attributes:
    id:             total_precipitation_sum
    data_type:      {'type': 'PixelType', 'precision': 'double'}
    crs:            EPSG:4326
    crs_transform:  [0.1, 0, -180.05, 0, -0.1, 90.05]

In [ ]:
anomalia

<xarray.DataArray 'total_precipitation_sum' (time: 1, lat: 355, lon: 410)> Size: 582kB
array([[[nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        ...,
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan],
        [nan, nan, nan, ..., nan, nan, nan]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 8B 2023-12-01
  * lon      (lon) float64 3kB -73.93 -73.82 -73.71 ... -29.16 -29.05 -28.94
  * lat      (lat) float64 3kB -33.7 -33.59 -33.48 -33.37 ... 5.025 5.135 5.245
    month    (time) int64 8B 12
Attributes:
    pyproj_srs:  +proj=longlat +datum=WGS84 +no_defs

In [ ]:
#ds_precip_mensal['total_precipitation_sum'][0,:,:].plot(x='lon', y='lat', cmap='jet')

In [ ]:
#ds_precip_mensal['total_precipitation_sum'].loc['2025-07-01'].plot(x='lon', y='lat', cmap='jet')

# Mapa de `TEMPERATURA` e `ANOMALIA`  

In [ ]:
%%time
# cria a moldura da figura
fig, ax = uplt.subplots(axwidth=6, ncols=2, nrows=1, tight=True, proj='pcarree')

# define formato da figura
ax.format(coast=False, borders=False, innerborders=False,
          labels=False, latlines=5, lonlines=10,
          latlim=(latmin, latmax), lonlim=(lonmin, lonmax),
          small='20px', large='25px',
          linewidth=0, grid=False)

# plota subtítulo
ax.text(lonmin, latmax-1.0, f'{nome_mes[int(mes)-1]} de {ano}', color='grey', fontsize=10)

#--------------------------------------------------------------#
#                     FIG-1: PRECIPITAÇÃO
#--------------------------------------------------------------#
# plota mapa
map1 = ax[0].contourf(ds_precip_mensal['lon'],
                      ds_precip_mensal['lat'],
                      ds_precip_mensal['total_precipitation_sum'].loc[f'{ano}-{mes}-01'],
                      cmap='oslo_r',
                      vmin=50, vmax=400,
                      levels=uplt.arange(50, 400, 50),
                      extend='both',
                      colorbar='best',
                      colorbar_kw={'label': 'Fonte: ERA5-Land Monthly/Pixel: 11km',
                                   'length': 18,
                                   'frameon': False,
                                   'ticklabelsize': 10,
                                   'labelsize': 7,
                                   'width': 2,
                                   'ticks': 50,
                                   'minorticks': False})

# título da figura
ax[0].format(title=f'Acumulado de Precipitação (mm)',
             titleloc='l',
             titleweight='bold',
             titlecolor='black')

# plota contorno dos Estados
estados_brasil = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/estados_do_brasil/BR_UF_2019.shp')
estados_brasil.plot(edgecolor='gray', facecolor='none', linewidth=0.5, alpha=1, ax=ax[0])

# plota contorno do Brasil
contorno_brasil = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/brasil/BRAZIL.shp')
contorno_brasil.plot(edgecolor='black', facecolor='none', linewidth=1.0, alpha=1, ax=ax[0])

# informação na figura
ax[0].annotate('Prof. Enrique Mattos/UNIFEI\ngithub.com/evmpython', xy=(lonmin,latmin), fontsize=7, color='black')

#--------------------------------------------------------------#
#                     FIG-2: ANOMALIA
#--------------------------------------------------------------#
# plota mapa
ax[1].contourf(anomalia['lon'],
               anomalia['lat'],
               anomalia[0,:,:],
               cmap='DryWet',
               vmin=-125, vmax=125,
               levels=uplt.arange(-125, 125, 25),
               extend='both',
               colorbar='best',
               colorbar_kw={'label': 'Fonte: ERA5-Land Monthly/Pixel: 11km',
                            'length': 18,
                            'frameon': False,
                            'ticklabelsize': 10,
                            'labelsize': 7,
                            'width': 2,
                            'ticks': 50,
                            'minorticks': False})

# título da figura
ax[1].format(title=f'Anomalia de Precipitação (mm)',
             titleloc='l',
             titleweight='bold',
             titlecolor='black')

# plota contorno dos Estados
estados_brasil = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/estados_do_brasil/BR_UF_2019.shp')
estados_brasil.plot(edgecolor='gray', facecolor='none', linewidth=0.5, alpha=1, ax=ax[1])

# plota contorno do Brasil
contorno_brasil = salem.read_shapefile('https://github.com/evmpython/shapefile/raw/main/brasil/BRAZIL.shp')
contorno_brasil.plot(edgecolor='black', facecolor='none', linewidth=1.0, alpha=1, ax=ax[1])

# salva figura
fig.save(f'Fig_02_ERA5_precipitacao_e_anomalia_{ano}-{mes}.png', dpi=300)